# DST-10_Project3_Arcis_Predict TripAdvisor Rating
В данном проекте нам предстоит предсказать рейтинг ресторана в TripAdvisor


# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

df = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
df.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

# Restaurant_id

Идентификатор ресторана. Не интересен с точки зрения обучения модели. Пропусков нет.

# City

In [ ]:
df_train['City'].value_counts(dropna = False, ascending=True).plot(kind='barh')

In [ ]:
train_city = set(df_train['City'].value_counts().index)
test_city = set(df_test['City'].value_counts().index)
test_city - train_city

**В тестовом наборе данных содержатся те же города что и в обучающей выборке**

Давайте найдём квантили 0.9 и 0.1 для рейтинга по каждому городу и добавим эту информацию в соотвествующие столбцы.

In [ ]:
def city_rating_quantile_mean(df):
    
    quantile_09 = df_train.groupby(['City'])['Rating'].quantile(0.9)
    quantile_01 = df_train.groupby(['City'])['Rating'].quantile(0.1)
    mean_city = df_train.groupby(['City'])['Rating'].mean()
    
    df['city_rating_quantile_09'] = df['City'].apply(lambda x: quantile_09[x])
    df['city_rating_quantile_01'] = df['City'].apply(lambda x: quantile_01[x])
    df['city_rating_mean'] = df['City'].apply(lambda x: mean_city[x])
    
    return None    

In [ ]:
def city_dummies(df):
    df['City2'] = df['City']
   
    return pd.get_dummies(df, columns=[ 'City2',], dummy_na=True)

## Команды для Preprocessing


In [ ]:
city_rating_quantile_mean(df)
df = city_dummies(df)

## Резюме по параметру City: 
- категориальный признак
- пропусков нет
- параметр перекодирован по типу dummies
- сгенерированы новые признаки в связке с rating:
    - city_rating_quantile_09
    - city_rating_quantile_01
    - city_rating_mean

# Cuisine Style

In [ ]:
df['Cuisine Style'][0]

Тут явно требуется предобработка для того чтобы список кухонь стал именно списком. Вернёмся к этому чуть позже

In [ ]:
pd.isna(df['Cuisine Style']).sum()/50000

Более 23% пропусков, давайте закодируем их в отдельный признак

In [ ]:
def cuisine_style_nans(df):
    df['cuisine_style_nans'] = pd.isna(df['Cuisine Style']).astype('uint8')
    return None

In [ ]:
def cuisine_list(x):
    if type(x) is str:
        return x.strip("[]").replace("'", '').split(', ')
    else:
        return ['other'] # Заменим пустые списки на отдельный тип кухни - 'other'

In [ ]:
def cuisine_count():
    cuisine_dict = {}
    for row in df['Cuisine Style']:
            if len(row) > 0:
                for i, v in enumerate(row):
                    cuisine_dict.setdefault(row[i], 0)
                    cuisine_dict[row[i]]+=1
    df_cuisine_style = pd.DataFrame.from_dict(cuisine_dict, orient='index', columns = ['count'])
    df_cuisine_style.sort_values(by='count', ascending=False, inplace = True)
    df_cuisine_style.reset_index(inplace = True)
    df_cuisine_style.columns = ['cuisine', 'count']
    
    return df_cuisine_style # возвращаем датафрейм с кухнями и их количеством по всем ресторанам

Пока оставим эту информацию для разработки инсайтов позже

## Команды для Preprocessing

In [ ]:
cuisine_style_nans(df) # генерация нового признака - кол.-во пропусков
df['Cuisine Style'] = df['Cuisine Style'].apply(cuisine_list) # преобразование в списки
df['count_cuisine_style'] = df['Cuisine Style'].apply(lambda x: len(x)).astype('uint8') # ген. нового признака   


## Резюме по параметру Cuisine Style

- категориальный признак
- пропусков нет
- параметр перекодирован по типу dummies
- сгенерированы следующие параметры:
    - count_cuisine_style - количество кухонь в ресторане
    - cuisine_style_nans - индикатор отстуствия информации по кухням

# Ranking

In [ ]:
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

Очевидно, что данный параметр напрямую зависит от количества ресторанов в городе. Чтобы этот параметр можно было соотносить с ranking в других города этот параметр нужно нормировать.

In [ ]:
def norm_Ranking(df):
    # найдём для каждого города максимальный и средний Ranking
    max_ranking_on_city = df.groupby(['City'])['Ranking'].max() 
    mean_ranking_on_city = df.groupby(['City'])['Ranking'].mean()
    std_ranking_on_city = df.groupby(['City'])['Ranking'].std()
    
    # в каждой строке пропишем соотвествующее значение максимального и среднего Ranking
    df['max_ranking_on_city'] = df['City'].apply(lambda x: max_ranking_on_city[x])
    df['mean_ranking_on_city'] = df['City'].apply(lambda x: mean_ranking_on_city[x])
    
    
    df['std_restorant_in_city'] = df['City'].apply(lambda x: std_ranking_on_city[x])
    
    df['norm_ranking_min_max'] = df['Ranking'] / df['max_ranking_on_city']
    df['norm_ranking_std'] = (df['Ranking'] - df['mean_ranking_on_city']) / df['std_restorant_in_city']
    return None

## Команды для Preprocessing

In [ ]:
norm_Ranking(df)

## Резюме по параметру Ranking

- числовой признак (выражен натуральными числами)
- пропусков нет
- сгенерированы признаки:
    - нормализованные параметры ranking (minmax и std)
    - max_ranking_on_city - количество ресторанов в городе
    - mean_ranking_on_city - средний рейтинг в городе
    - std_restorant_in_city - стандартное отклонение в рейтинга в городе
    

# Price Range

In [ ]:
df['Price Range'].value_counts(dropna = False)

Оценка стоимости среднего чека может быть заменена на числа: 1, 2, 3.

Пропуски заменим пока средним значением - 2. 

In [ ]:
def price_range(df):
    df['Price Range'].fillna(2, inplace = True)
    df['Price Range'].replace(['$', '$$ - $$$', '$$$$'], [1, 2, 3], inplace = True)
    return df['Price Range']

## Команды для Preprocessing

In [ ]:
df['Price Range'] = price_range(df)

# Number of Reviews

В данном параметре есть пропуски - окло 6%. Сохраним эту информаци в виде отдельного признака.
А пропуски заполним нулями

## Команды для Preprocessing

In [ ]:
df['number_of_reviews_nans'] = pd.isna(df['Number of Reviews']).astype('uint8')
data['Number of Reviews'].fillna(0, inplace=True)

# Reviews

# URL_TA

Адрес странички рестарана на сайте TripAdvisor.
Не думаю, что тут есть полезные для нас данные.

# ID_TA

Очередной идентификатор, также не является полезным для нас.

# Rating - целевая переменная

In [ ]:
df['Rating'].hist()

0 - это тестовая часть данных (их ровно 10 000)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    df = df_input.copy()
    
    ### ПРЕДОБРАБОТКА И СОЗДАНИЕ НОВЫХ ПАРАМЕТРОВ ###
    
       #-->  1. Сity  <--#
        
    city_rating_quantile_mean(df)
    df = city_dummies(df)

       #--> 2. Cuisine Style <--#
    
    cuisine_style_nans(df) # генерация нового признака - кол.-во пропусков
    df['Cuisine Style'] = df['Cuisine Style'].apply(cuisine_list) # преобразование в списки
    # ген. нового признака - количетво кухонь в ресторане
    df['count_cuisine_style'] = df['Cuisine Style'].apply(lambda x: len(x)).astype('uint8')
       
       #--> 3. Ranking <--#
    
    norm_Ranking(df)
    
       #--> 4. Price Range <--#
        
    df['Price Range'] = price_range(df)
        
       #--> 5. Number of Reviews <--#
    
    df['number_of_reviews_nans'] = pd.isna(df['Number of Reviews']).astype('uint8')
    df['Number of Reviews'].fillna(0, inplace=True)
    
       #--> 6. Reviews <--#
        
       #-->  <--#
    
    ### УДАЛЯЕМ ВСЕ НЕЧИСЛОВЫЕ ПАРАМЕТРЫ ###
    
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df.columns if df[s].dtypes == 'object']
    df.drop(object_columns, axis = 1, inplace=True)
    
    print('Удалили эти столбцы -> ', object_columns)

    return df

df_preproc = preproc_data(data)
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

'''
Перед тем как отправлять наши данные на обучение, 
разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, 
до отправки submissiona на kaggle.
'''

# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)
y_pred = np.around(y_pred/0.5, decimals=0, out=None)*0.5


# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
# Переобучим модель на всех данных
model.fit(X, y)

#test_data = test_data.drop(['Rating'], axis=1)

predict_submission = model.predict(test_data)
predict_submission = np.around(predict_submission/0.5, decimals=0, out=None)*0.5

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
